In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yiweilu2033/well-documented-alzheimers-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/well-documented-alzheimers-dataset


In [ ]:
import os
print(os.listdir(path))

['ModerateDemented', 'NonDemented (2)', 'oasis_cross-sectional-5708aa0a98d82080 (1).xlsx', 'VeryMildDemented', 'MildDemented']


In [ ]:
import os

def count_images_in_folders(dataset_path):
    folder_counts = {}

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)

        if os.path.isdir(folder_path):
            # Buscar subcarpeta con el mismo nombre (excepto para 'NonDemented')
            subfolder_name = folder if "NonDemented" not in folder else "NonDemented"
            subfolder_path = os.path.join(folder_path, subfolder_name)

            if os.path.isdir(subfolder_path):
                image_count = len([f for f in os.listdir(subfolder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))])
                folder_counts[folder] = image_count
            else:
               print(f"Subcarpeta no encontrada en: {folder_path}")

    return folder_counts

# Ruta del dataset

image_counts = count_images_in_folders(path)

# Imprimir resultados
for folder, count in image_counts.items():
    print(f"{folder}: {count} imágenes")


ModerateDemented: 376 imágenes
NonDemented (2): 63560 imágenes
VeryMildDemented: 13796 imágenes
MildDemented: 5184 imágenes


In [ ]:
import os

def count_images_in_folders(dataset_path):
    folder_counts = {}

    for folder in os.listdir(dataset_path):
        # Omitir la carpeta "ModerateDemented"
        if folder == "ModerateDemented":
            continue

        folder_path = os.path.join(dataset_path, folder)

        if os.path.isdir(folder_path):
            # Buscar subcarpeta con el mismo nombre (excepto para 'NonDemented')
            subfolder_name = folder if "NonDemented" not in folder else "NonDemented"
            subfolder_path = os.path.join(folder_path, subfolder_name)

            if os.path.isdir(subfolder_path):
                image_count = len([f for f in os.listdir(subfolder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))])
                folder_counts[folder] = image_count
            else:
                print(f"Subcarpeta no encontrada en: {folder_path}")

    return folder_counts

# Ruta del dataset

image_counts = count_images_in_folders(path)

# Imprimir resultados
for folder, count in image_counts.items():
    print(f"{folder}: {count} imágenes")


NonDemented (2): 63560 imágenes
VeryMildDemented: 13796 imágenes
MildDemented: 5184 imágenes


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Directorio base donde están las imágenes
base_dir =path

# Diccionario con la estructura correcta de carpetas y subcarpetas (sin "ModerateDemented")
category_subfolders = {
    "MildDemented": "MildDemented",
    "VeryMildDemented": "VeryMildDemented",
    "NonDemented (2)": "NonDemented"
}

# Crear listas para almacenar las rutas de las imágenes, los IDs de los pacientes y sus etiquetas
image_paths = []
patient_ids = []
labels = []

# Recorrer cada carpeta y buscar imágenes en la subcarpeta correspondiente
for category, subfolder in category_subfolders.items():
    category_path = os.path.join(base_dir, category, subfolder)  # Ruta correcta a la subcarpeta

    if not os.path.exists(category_path):
        print(f"Advertencia: No se encontró la carpeta {category_path}, se omitirá.")
        continue  # Si la carpeta no existe, pasar a la siguiente

    for img_name in os.listdir(category_path):
        if img_name.lower().endswith((".png", ".jpg", ".jpeg")):  # Asegurar que es una imagen

            # Extraer ID del nombre de la imagen (tomando los primeros 3 segmentos del nombre de archivo)
            patient_id = "_".join(img_name.split("_")[:3])

            # Almacenar la información del paciente, la ruta de la imagen y su etiqueta
            image_paths.append(os.path.join(category_path, img_name))
            patient_ids.append(patient_id)
            labels.append(category)

# Crear DataFrame con la información de las imágenes
images_df = pd.DataFrame({
    "image_path": image_paths,
    "patient_id": patient_ids,
    "label": labels
})

# Obtener las etiquetas de los pacientes para la estratificación
patient_labels = images_df.groupby('patient_id')['label'].first()  # Asignar la primera etiqueta de cada paciente

# Dividir los pacientes en conjunto de entrenamiento (80%), validación (10%) y prueba (10%)
# Primero obtenemos los IDs únicos de los pacientes
unique_patient_ids = images_df["patient_id"].unique()

# Dividir los pacientes en entrenamiento (80%) y prueba (20%)
train_patient_ids, test_patient_ids = train_test_split(
    unique_patient_ids, test_size=0.2, random_state=42, stratify=patient_labels.loc[unique_patient_ids]
)

# Dividir el conjunto de entrenamiento en entrenamiento (80%) y validación (20%)
train_patient_ids, val_patient_ids = train_test_split(
    train_patient_ids, test_size=0.2, random_state=42, stratify=patient_labels.loc[train_patient_ids]
)

# Filtrar las imágenes correspondientes a cada conjunto de pacientes
train_data = images_df[images_df["patient_id"].isin(train_patient_ids)]
val_data = images_df[images_df["patient_id"].isin(val_patient_ids)]
test_data = images_df[images_df["patient_id"].isin(test_patient_ids)]

# Verificar el tamaño de cada conjunto
print(f"Tamaño de entrenamiento: {len(train_data)}")
print(f"Tamaño de validación: {len(val_data)}")
print(f"Tamaño de prueba: {len(test_data)}")

# Opcional: crear un DataFrame final con la información de las imágenes y los conjuntos
final_df = pd.DataFrame({
    "image_path": image_paths,
    "patient_id": patient_ids,
    "label": labels,
    "set": ['train' if pid in train_patient_ids else ('val' if pid in val_patient_ids else 'test') for pid in patient_ids]
})

# Verificar el DataFrame final
print(final_df.head())


Tamaño de entrenamiento: 52331
Tamaño de validación: 13293
Tamaño de prueba: 16916
                                          image_path     patient_id  \
0  /kaggle/input/well-documented-alzheimers-datas...  OAS1_0073_MR1   
1  /kaggle/input/well-documented-alzheimers-datas...  OAS1_0316_MR1   
2  /kaggle/input/well-documented-alzheimers-datas...  OAS1_0430_MR1   
3  /kaggle/input/well-documented-alzheimers-datas...  OAS1_0184_MR1   
4  /kaggle/input/well-documented-alzheimers-datas...  OAS1_0269_MR1   

          label    set  
0  MildDemented  train  
1  MildDemented  train  
2  MildDemented  train  
3  MildDemented  train  
4  MildDemented  train  


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Transformaciones (iguales para todos)
transform = transforms.Compose([
    transforms.Resize((224, 224)),                 # Redimensionar la imagen
    transforms.RandomHorizontalFlip(p=0.5),        # Volteo aleatorio horizontal
    transforms.RandomRotation(degrees=15),         # Rotación aleatoria
    transforms.ToTensor(),                         # Convertir la imagen a tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalización
])

# Mapeo de etiquetas a números (sin "ModerateDemented")
label_map = {"MildDemented": 0, "VeryMildDemented": 1, "NonDemented (2)": 2}

class AlzheimerDataset(Dataset):
    def __init__(self, df, transform=None):
        """
        Constructor de la clase que inicializa el dataset de Alzheimer.

        Args:
        df (pd.DataFrame): DataFrame con las rutas de imágenes y las etiquetas.
        transform (callable, optional): Transformaciones a aplicar a cada imagen.
        """
        self.df = df
        self.transform = transform

    def __len__(self):
        """Devuelve el número total de muestras (imágenes) en el dataset."""
        return len(self.df)

    def __getitem__(self, idx):
        """
        Obtiene una imagen y su etiqueta correspondiente del DataFrame.

        Args:
        idx (int): Índice de la muestra.

        Returns:
        image (Tensor): Imagen transformada.
        label (int): Etiqueta correspondiente a la imagen.
        """
        img_path = self.df.iloc[idx]["image_path"]
        label = label_map[self.df.iloc[idx]["label"]]

        # Cargar la imagen
        image = Image.open(img_path).convert("RGB")

        # Aplicar transformaciones (si las hay)
        if self.transform:
            image = self.transform(image)

        return image, label

# Filtrar el DataFrame para excluir "ModerateDemented"
filtered_df = final_df[final_df["label"] != "ModerateDemented"]

# Filtramos los DataFrames para los conjuntos de entrenamiento, validación y prueba
train_df = filtered_df[filtered_df["set"] == "train"]
val_df = filtered_df[filtered_df["set"] == "val"]
test_df = filtered_df[filtered_df["set"] == "test"]

# Crear los datasets de PyTorch
train_dataset = AlzheimerDataset(train_df, transform=transform)
val_dataset = AlzheimerDataset(val_df, transform=transform)
test_dataset = AlzheimerDataset(test_df, transform=transform)

# Verificar el tamaño de cada dataset
print(f"Tamaño del dataset -> Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

# Crear los DataLoaders para cargar los datos por lotes
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"DataLoaders creados con batch_size={batch_size}")


Tamaño del dataset -> Train: 52331, Val: 13293, Test: 16916
DataLoaders creados con batch_size=128


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
from torchvision import models

class PaperAlzheimerNet(nn.Module):
    def __init__(self, num_classes=3):
        super(PaperAlzheimerNet, self).__init__()

        # Cargar MobileNetV2 preentrenado
        mobilenet = models.mobilenet_v2(weights='IMAGENET1K_V1')

        # Congelar capas si quieres transfer learning parcial
        for param in mobilenet.features.parameters():
            param.requires_grad = True  # o False si quieres congelar

        # Usar solo las capas convolucionales (sin la clasificación)
        self.features = mobilenet.features  # Output shape: (batch, 1280, 7, 7)

        # Clasificador personalizado basado en el paper
        self.classifier = nn.Sequential(
            nn.Flatten(),                       # -> (batch, 1280*7*7) = 62720
            nn.Linear(62720, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# Inicializar modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PaperAlzheimerNet(num_classes=3).to(device)

print(model)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 130MB/s]


PaperAlzheimerNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from google.colab import files

# 1. Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 2. Función para entrenar el modelo (SIN Early Stopping)
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    train_losses = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels)
            total_preds += labels.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct_preds / total_preds

        # Evaluación en validación
        model.eval()
        running_val_loss = 0.0
        correct_preds_val = 0
        total_preds_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_val_loss += loss.item() * inputs.size(0)

                _, preds = torch.max(outputs, 1)
                correct_preds_val += torch.sum(preds == labels)
                total_preds_val += labels.size(0)

        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        epoch_val_acc = correct_preds_val / total_preds_val

        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        # Guardar el mejor modelo basado en la pérdida de validación
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            torch.save(model.state_dict(), "best_model.pth")
            print(f" Mejor modelo guardado en la época {epoch+1} con val_loss: {epoch_val_loss:.4f}")

        # Imprimir resultados de la época
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {epoch_train_acc:.4f} - "
              f"Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {epoch_val_acc:.4f}")

    return train_losses, val_losses, val_accuracies

# 3. Función para evaluar el modelo en el conjunto de prueba
def evaluate_model(model, test_loader):
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    correct_preds_test = 0
    total_preds_test = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            correct_preds_test += torch.sum(preds == labels)
            total_preds_test += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_accuracy = correct_preds_test / total_preds_test
    print(f"Test Accuracy: {test_accuracy:.4f}")

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy (sklearn): {accuracy:.4f}")

    return accuracy

# 4. Entrenar el modelo (SIN Early Stopping)
num_epochs = 25
train_losses, val_losses, val_accuracies = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

# 5. Evaluación final en el conjunto de prueba
test_accuracy = evaluate_model(model, test_loader)

# 6. Guardar el mejor modelo entrenado
model_save_path = "modelopaper187-3class.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Modelo guardado en {model_save_path}")
files.download(model_save_path)


 Mejor modelo guardado en la época 1 con val_loss: 0.5698
Epoch 1/25 - Train Loss: 0.3808, Train Accuracy: 0.8218 - Validation Loss: 0.5698, Validation Accuracy: 0.7808
Epoch 2/25 - Train Loss: 0.2635, Train Accuracy: 0.8880 - Validation Loss: 0.8381, Validation Accuracy: 0.7713
Epoch 3/25 - Train Loss: 0.1819, Train Accuracy: 0.9279 - Validation Loss: 0.8222, Validation Accuracy: 0.7274
Epoch 4/25 - Train Loss: 0.1324, Train Accuracy: 0.9496 - Validation Loss: 1.0368, Validation Accuracy: 0.7590
Epoch 5/25 - Train Loss: 0.0987, Train Accuracy: 0.9626 - Validation Loss: 1.2200, Validation Accuracy: 0.7231
Epoch 6/25 - Train Loss: 0.0758, Train Accuracy: 0.9724 - Validation Loss: 1.1629, Validation Accuracy: 0.7481
Epoch 7/25 - Train Loss: 0.0658, Train Accuracy: 0.9762 - Validation Loss: 1.0131, Validation Accuracy: 0.7797
Epoch 8/25 - Train Loss: 0.0582, Train Accuracy: 0.9789 - Validation Loss: 1.4316, Validation Accuracy: 0.7530
Epoch 9/25 - Train Loss: 0.0509, Train Accuracy: 0.982

KeyboardInterrupt: 

In [ ]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Evaluar el modelo en el conjunto de prueba
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Crear la matriz de confusión
labels_classes = ["MildDemented", "VeryMildDemented", "NonDemented"]  # Eliminado "ModerateDemented"
cm = confusion_matrix(y_true, y_pred)

# Graficar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels_classes, yticklabels=labels_classes)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Matriz de Confusión")
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np
import matplotlib.pyplot as plt

# Convertir las etiquetas en formato binario (one-vs-all) para 3 clases
y_true_bin = label_binarize(y_true, classes=[0, 1, 2])  # Solo 3 clases ahora
y_scores = []

with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)  # Obtiene los logits (valores antes de softmax)
        y_scores.extend(outputs.cpu().numpy())

y_scores = np.array(y_scores)

# Graficar la curva ROC para cada clase
plt.figure(figsize=(8,6))
labels_classes = ["MildDemented", "VeryMildDemented", "NonDemented"]  # Sin "ModerateDemented"

for i in range(3):  # Ahora solo 3 clases
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Clase {labels_classes[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')  # Línea diagonal
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curvas ROC por Clase")
plt.legend()
plt.show()
